##Khalid NASER ##



In [ ]:
# Importing packages
import json
import pymongo
from bson import ObjectId

client = pymongo.MongoClient('localhost', 27017)
# create db tutorial
mydb = client["homework"]
# create collection example
collection = mydb["pubmed"]
with open('Chap3/pubmed_cleaned.json') as file:
    file_data = json.load(file)

In [ ]:
for document in file_data:
    if '_id' in document and '$oid'in document['_id']:
        document['_id']= ObjectId(document['_id']['$oid'])
collection.insert_many(file_data)    

In [ ]:
#1)Create an index, explain your choice of key.
collection.create_index([ ("pmid",1) ])
# the choice of indexing pmid field is because that's the field that i'm gonna search frequently in this exercice.

'pmid_1'

In [ ]:
#2)Delete every paper that was published prior 2019
result = collection.delete_many({"date": {"$lt": "year 2019, month 1, day 1, hour 0, minute 0"}})
print(f"Nombre de documents supprimés: {result.deleted_count}")

Nombre de documents supprimés: 5


In [ ]:
num_single_author_papers = collection.count_documents({"authors": {"$regex": "name ml"}})
print(f"Nombre de documents avec un seul auteur : {num_single_author_papers}")

Nombre de documents avec un seul auteur : 99409


In [ ]:
# 3)How many paper have a single author ? Two authors ?
num_single_author_documents = 0
num_two_authors_documents = 0

# Count The number of paper that have one or two authors
for document in collection.find():
    authors = document['authors'].split('\n')
    num_authors = len(authors)
    if num_authors == 1:
        num_single_author_documents += 1
    elif num_authors == 2:
        num_two_authors_documents += 1

# Print results
print("Nombre de documents avec un seul auteur :", num_single_author_documents)
print("Nombre de documents avec deux auteurs :", num_two_authors_documents)

Nombre de documents avec un seul auteur : 5480
Nombre de documents avec deux auteurs : 10153


In [ ]:
#4)What's the last paper inserted in the db ?
x = collection.find_one(sort=[('_id', -1)])
print("Last Paper:")
print("Title:", last_paper['title'])
print("Authors:", last_paper['authors'])
print("Date:", last_paper['date'])

In [ ]:
#5)Find articles with null meshwords.
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['homework']
collection = db['pubmed']

# Find articles with null meshwords
articles_with_null_meshwords = collection.find({"meshwords": {"$in": [None, ""]}})

# Print the details of articles with null meshwords
print("Articles with null meshwords:")
for article in articles_with_null_meshwords:
    print("Title:", article['title'])

In [ ]:
#6)Choose a keyword you are interested in (machine learning, computer vision,...). 
#Find the number of articles with the choosen keyword in their meshwords, abstract or title.
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['homework']
collection = db['pubmed']
keyword = "machine learning"

# Count the number of articles with the keyword in meshwords, abstract, or title
article_count = collection.count_documents({
    "$or": [
        {"meshwords": {"$regex": f".*{keyword}.*", "$options": "i"}},
        {"abstract": {"$regex": f".*{keyword}.*", "$options": "i"}},
        {"title": {"$regex": f".*{keyword}.*", "$options": "i"}}
    ]
})

print(f"Number of articles with the keyword '{keyword}': {article_count}")

Number of articles with the keyword 'machine learning': 521


In [ ]:
#7)What's the number of articles that have atleast one affiliation AND meshwords.
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['homework']
collection = db['pubmed']

article_count = collection.count_documents({
    "$and": [
        {"authors": {"$ne": None}},
        {"meshwords": {"$ne": None}}
    ]
})

print(f"Number of articles with at least one affiliation and meshwords: {article_count}")

Number of articles with at least one affiliation and meshwords: 75314


In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['homework']
collection = db['pubmed']

In [ ]:
# 8)Count the number of articles with a publishing date after 2020
article_count = collection.count_documents({
    "date": {"$gt": "year 2020"}
})

print(f"Number of articles with a publishing date after 2020: {article_count}")

Number of articles with a publishing date after 2020: 0


In [ ]:
#9)Find articles where there's atleast one affiliation from a choosen country (you decide which one).
# Count the number of articles with at least one affiliation from Canada
article_count = collection.count_documents({
    "authors": {"$regex": "Canada"}})
print("Number of articles with at least one affiliation from Canada:", article_count)
article_country= collection.find({"authors":{"$regex": "Canada"}})
for article in article_country:
    print("ID:", article["_id"],"Title :", article["title"])


In [ ]:
#10)Check for any duplicates
distinct_pmids = collection.distinct("pmid")
# Check for duplicates
duplicates = len(distinct_pmids) != collection.count_documents({})
# Print the result
if duplicates:
    print("Duplicates found.")
else:
    print("No duplicates found.")

No duplicates found.


In [ ]:
#11)Remove every articles where the abstract starts with an "R".
print("init count: ",collection.count_documents({"abstract": {"$regex": "^\\s*\"R"}}))
collection.delete_many({"abstract": {"$regex": "^\\s*\"R"}})
print("after delete many: ",collection.count_documents({"abstract": {"$regex": "^\\s*\"R"}}))

init count:  1918
after delete many:  0


In [ ]:
#12)Return the list of papers (pmid) where there's atleast one affiliation per author
papers = collection.find(
    {"authors": {"$exists": True}}
)
paper_list = [paper["pmid"] for paper in papers if len(paper.get("authors", [])) > 0]
print(paper_list)